In [1]:
import traffic
import pandas as pd 
import os
from traffic.core import Traffic
from biblio_herramienta.herramienta import *
from biblio_herramienta.tratardatos import *
from biblio_herramienta.conflictos import *

In [2]:
carpetaDatos = r'datos_sectores/' # carpeta que contiene los datos
#nombreDatos = r"bilbao_f_cluster.csv"
nombreDatos = r"datos_javi_filtrado.csv"
carpetaImagenes = r"generacion_conflictos/"

In [3]:
vuelos = cargardatos(carpetaDatos,nombreDatos)

In [15]:
vuelos.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66743 entries, 0 to 66742
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Unnamed: 0       66743 non-null  int64              
 1   Unnamed: 0.1     66743 non-null  int64              
 2   alert            66743 non-null  bool               
 3   altitude         66743 non-null  float64            
 4   callsign         66743 non-null  object             
 5   geoaltitude      66743 non-null  float64            
 6   groundspeed      66743 non-null  float64            
 7   hour             66743 non-null  datetime64[ns, UTC]
 8   icao24           66743 non-null  object             
 9   last_position    66743 non-null  datetime64[ns, UTC]
 10  latitude         66743 non-null  float64            
 11  longitude        66743 non-null  float64            
 12  onground         66743 non-null  bool               
 13  spi             

In [14]:
CPA = vuelos.closest_point_of_approach(lateral_separation = 10*1852, vertical_separation = 2000)



/Users/jaimebowen/opt/miniconda3/envs/tfgpaquetes/lib/python3.7/site-packages/traffic/algorithms/cpa.py:151: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Defaulting to projection EuroPP()")


/Users/jaimebowen/opt/miniconda3/envs/tfgpaquetes/lib/python3.7/site-packages/traffic/core/mixins.py:358: UserWarning: The chosen projection is invalid for current shape
  warnings.warn("The chosen projection is invalid for current shape")


/Users/jaimebowen/opt/miniconda3/envs/tfgpaquetes/lib/python3.7/site-packages/traffic/core/mixins.py:358: UserWarning: The chosen projection is invalid for current shape
  warnings.warn("The chosen projection is invalid for current shape")
/Users/jaimebowen/opt/miniconda3/envs/tfgpaquetes/lib/python3.7/site-packages/traffic/core/mixins.py:358: UserWarning: The chosen projection is invalid for current shape
  warnings.warn("The chosen projection is invalid for current shape")


In [5]:
CPA 

,timestamp,latitude_x,longitude_x,altitude_x,icao24_x,callsign_x,flight_id_x,latitude_y,longitude_y,altitude_y,icao24_y,callsign_y,flight_id_y,lateral,vertical
0,2019-01-01 10:41:06+00:00,43.089915,-2.466234,33000.0,aacaa5,AAL67,AAL67_264,43.219757,-2.842201,36000.0,4cab71,RYR8XV,RYR8XV_253,18.256508,3000.0
1,2019-01-01 10:41:07+00:00,43.091125,-2.468850,33000.0,aacaa5,AAL67,AAL67_264,43.220960,-2.841993,36000.0,4cab71,RYR8XV,RYR8XV_253,18.143920,3000.0
2,2019-01-01 10:41:08+00:00,43.091721,-2.470135,33000.0,aacaa5,AAL67,AAL67_264,43.223007,-2.841690,36000.0,4cab71,RYR8XV,RYR8XV_253,18.118351,3000.0
3,2019-01-01 10:41:09+00:00,43.093546,-2.474082,33000.0,aacaa5,AAL67,AAL67_264,43.225056,-2.841405,36000.0,4cab71,RYR8XV,RYR8XV_253,17.956568,3000.0
4,2019-01-01 10:41:10+00:00,43.095016,-2.477289,33000.0,aacaa5,AAL67,AAL67_264,43.225056,-2.841405,36000.0,4cab71,RYR8XV,RYR8XV_253,17.791127,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2019-01-02 13:16:55+00:00,43.069991,-3.103507,34000.0,3452c9,VLG3971,VLG3971_089,43.268005,-2.946955,36000.0,342091,IBE31GL,IBE31GL_014,13.723846,2000.0
24,2019-01-02 13:16:56+00:00,43.069199,-3.100760,34000.0,3452c9,VLG3971,VLG3971_089,43.270752,-2.946061,36000.0,342091,IBE31GL,IBE31GL_014,13.867876,2000.0
25,2019-01-02 13:16:57+00:00,43.068466,-3.098145,34025.0,3452c9,VLG3971,VLG3971_089,43.272400,-2.945513,36000.0,342091,IBE31GL,IBE31GL_014,13.948702,1975.0
26,2019-01-02 13:16:58+00:00,43.068466,-3.098145,34000.0,3452c9,VLG3971,VLG3971_089,43.273424,-2.945251,36000.0,342091,IBE31GL,IBE31GL_014,14.008088,2000.0


In [6]:
def conflict_detection(CPA_datos):
    # Obtengo las filas en las que la mínima de separacion longitudinal es inferior a 5 MN
    rows_lon = CPA_datos['lateral'] < 5
    CPA_datos_lon = CPA_datos[rows_lon]
    CPA_datos_lon
    # De estas filas obtengo las que se cumple que es menor de 1000 ft la vertical
    rows_vert = CPA_datos_lon['vertical'] < 1000
    CPA_conf = CPA_datos_lon[rows_vert]
    if len(CPA_conf) > 0:
        conf = 1     # Nos da un 1 si hay alguna fila que cumpla ambas condiciones
        Minsep = min(CPA_conf['lateral'])
        # Para calcular el tiempo cogemos el primer instante en el que se genera el conflicto ya que la aeronave entra en el 0000, 
        # En el caso de que la aeronave no entrara en el instante 00 habría que restarlo
        # hora_original = Un problema ya que hay que coger el primer instante en el que entran las aeronaves
        dif_segundos = CPA_conf.iloc[0].timestamp.hour*3600 + CPA_conf.iloc[0].timestamp.minute*60 + CPA_conf.iloc[0].timestamp.second
        Timetoconf = dif_segundos
    else:
        conf = 0     # Nos da un 0 si no hay ninguna fila que cumpla una vulneración de ambas mínimas de separación
        # En el caso de que no haya conflictos ponemos un 10, no podemos poner la minima distancia ya  que quizá dos aeronaves se cruzan 
        #a 3 NM pero con una vertical de 30000 ft y por lo tanto se vuelve loco el algoritmo, también se podría pensar en ponerlo
        Minsep = 10
        # Lo mismo ocurrre con el tiempo
        Timetoconf = 10000 # Hemos puesto 10000 segundos pero sin saber realmente como puede afectar, habría que analizarlo
    return conf, Minsep, Timetoconf

In [9]:
def paresconconflictos(CPA):
    label_ave1, label_ave2 = pd.Series(CPA["flight_id_x"].unique()), pd.Series(CPA["flight_id_y"].unique())
    pairs_CPA = pd.DataFrame({'flight_id_x': label_ave1, 'flight_id_y': label_ave2})
    conflict = []    # Matriz en la que vamos a guardar si hay conflicto o no
    TTC = []         # Matriz en la que vamos a guardar el tiempo al conflicto
    MinDis = []      # Matriz en la que vamos a guardar la distancia mínima alcanzada entre aeronaves

    # Para cada una de las parejas tenemos que obtener la parte del CPA que le corrresponde:
    for index, row in pairs_CPA.iterrows():
        # Sacamos los nombres de cada par de aeronaves
        ave1, ave2 = row['flight_id_x'], row['flight_id_y']
        CPA_1 = CPA[CPA['flight_id_x'] == ave1]
        CPA_datos = CPA_1[CPA_1['flight_id_y'] == ave2]
        # CPA_datos contiene toda la información para un par de aeronaves con conflicto y le aplicamos la función conflicto
        conf, Minsep, Timetoconf = conflict_detection(CPA_datos)
        conflict.append(conf), MinDis.append(Minsep), TTC.append(Timetoconf)

    # Introducimos las variables de variación de altitud y de vertical speed
    pairs_CPA['Conflicto'] = conflict
    pairs_CPA['MinDis'] = MinDis
    pairs_CPA['Timetoconf'] = TTC
    return pairs_CPA
pairs_CPA = paresconconflictos(CPA)

In [13]:
pairs_CPA

,flight_id_x,flight_id_y,Conflicto,MinDis,Timetoconf
0,AAL67_264,RYR8XV_253,0,10.0,10000
1,EZY86MW_152,TOM2BY_156,0,10.0,10000
2,EZY454L_150,EZY86MW_152,0,10.0,10000
3,AZA63KD_200,EZY69QZ_136,0,10.0,10000
4,RYR8XV_253,RYR257_246,0,10.0,10000
...,...,...,...,...,...
86,BAW463_138,EUW9941_195,0,10.0,10000
87,VLG3971_089,RYR1CX_201,0,10.0,10000
88,NaN,RYR82VH_222,0,10.0,10000
89,NaN,ANE54HM_096,0,10.0,10000


In [8]:
# Una vez que tenemos todas aquellas aeronaves que son pareja y tienen los datos hay que incluir los indicadores
Datos_fil_con = Datos_fil
# Lo primero es generar una columna nueva que se llame conflictos en los datos filtrados que se rellene de 0
Datos_fil_con["Conflicto"] = 0
# Para la distancia ponemos un 10 como el peor caso
Datos_fil_con["MinDis"] = 10
# Para el tiempo ponemos un 10000 como hemos puesto en los otros para que no genere problema 
Datos_fil_con["Timetoconf"] = 10000

#Hay que construir una función que detecte las parejas de aeronaves que tienen un conflicto y cambien el 0 a un 1

NameError: name 'Datos_fil' is not defined

In [ ]:
def mod_conflictos(pairs_CPA, Datos_fil_con):
    # identificamos las filas que tienen conflicto
    Datos_final = Datos_fil_con
    rows_conflict = pairs_CPA['Conflicto'] == 1
    pairs_CPA_conflict = pairs_CPA[rows_conflict]
    
    # En el caso de que haya algún conflicto hay que cambiar los 0 por 1, y las distancias por sus valores 
    if len(pairs_CPA_conflict) > 0:
        i = 0
        for index, rows_conflict in pairs_CPA_conflict.iterrows():
            # identifico las aeronaves que tienen conflicto
            ave1_conflict, ave2_conflict = rows_conflict['flight_id_x'], rows_conflict['flight_id_y']
            # Obtenemos la row en la cual hay conflicto
            Datos_fil_con_ave1 = Datos_fil_con[Datos_fil_con['ave1'] == ave1_conflict]
            Datos_fil_con_ave1_ave2 = Datos_fil_con_ave1[Datos_fil_con_ave1['ave2'] == ave2_conflict]
            print(Datos_fil_con_ave1_ave2.index)
            # Identificados la row en la que hay conflicto, tenemos que cambiar el 1 por el 0
            Datos_final['Conflicto'][Datos_fil_con_ave1_ave2.index] = 1
            # Ahora calculamos la mínima distancia que le ocurre a este
            Minsep = pairs_CPA_conflict['MinDis'].iloc[i]
            Datos_final['MinDis'][Datos_fil_con_ave1_ave2.index] = Minsep
            # Lo mismo para el tiempo
            TTC = pairs_CPA_conflict['Timetoconf'].iloc[i]
            Datos_final['Timetoconf'][Datos_fil_con_ave1_ave2.index] = TTC
            # Pongo un contador para las matrices de distancia y tiempo
            i = i + 1
    else:
        Datos_final = Datos_fil_con
    
    return Datos_final

In [ ]:
BBDD = mod_conflictos(pairs_CPA, Datos_fil_con)
# Una vez introducidos los conflictos comprobamos que estas filas lo cumplen
BBDD.loc[(BBDD['Conflicto'] == 1)]

In [ ]:
BBDD_2 = BBDD
var_eliminar = ['ave1', 'ave2', 'groundspeed_1', 'timestamp_1', 'vertical_rate_1', 'track_1', 'altitude_2', 'geoaltitude_2',
                'latitude_2', 'longitude_2','timestamp_2','groundspeed_2','track_2','vertical_rate_2']
BBDD_2 = BBDD_2.drop(var_eliminar, axis = 'columns')

In [ ]:
BBDD_2.loc[(BBDD_2['Conflicto'] == 1)]

In [ ]:
# Vamos a determinar losflujos de tráfico en los que se producen cruces
con = BBDD_2.loc[(BBDD_2['Conflicto'] == 1)]
columnas = ['cluster_1', 'cluster_2']
# Podemos obtener los flujos que han presentado algún conflicto, se trata de valores únicos por lo que podríamos trabajar solo con estos
flujos_con_cruce = con[columnas].drop_duplicates()
# Por ejemplo si quiero trabajar con un flujo en particular tengo que seleccionar los valorers de una columna
Flujo_1 = flujos_con_cruce.iloc[0]
# Donde Flujo_1 es el vector que contiene los flujos a considerar y Flujo_1['cluster_1'] y Flujo_2['cluster_2'] nos indica a que flujos pertenecen

In [ ]:
BBDD_Flujos = BBDD_2.loc[BBDD_2['cluster_1'] == Flujo_1['cluster_1']]
BBDD_Flujos = BBDD_Flujos.loc[BBDD_2['cluster_2'] == Flujo_1['cluster_2']]
BBDD_Flujos['cluster_2']

In [ ]:
# Si no el siguiente paso sería obtener una matriz para cada uno de los flujos conflictos para que se automatice
# Aquí lo estamos calculando para las aeronaves del flujo 1 con el 0 por ejemplo, pero del 0 al 1 es otro flujo
j = 1
for i, flow in flujos_con_cruce.iterrows():
    # Seleccionamos de la BBDD las filas que cumplen el cluster_1
    filas_1 = BBDD_2.loc[BBDD_2['cluster_1'] == flow['cluster_1']]
    # Seleccionamos del primer filtrado las filas que cumplen el cluster_2
    filas_2 = filas_1.loc[filas_1['cluster_2'] == flow['cluster_2']]
    # Le damos un nombre con un número distinto a cada una de las matrices
    exec('Flujos{} = filas_2'.format(j))
    j = j + 1
# Con este paso ya hemos conseguido programar la subdivisión de los flujos para que lo haga de golpe y nos saquen tantas matrices como necesitamos